In [2]:
; cl-waffeを読み込む
(load "../../cl-waffe.asd")
; kfttを読み込むための自作ライブラリ
(load "../../examples/kftt-data-parser.lisp")
(ql:quickload `(:clgplot :cl-waffe) :silent t)

T

T

(:CLGPLOT :CL-WAFFE)

SB-INT:SIMPLE-STYLE-WARNING: undefined function: KFTT-DATA-PARSER::REGISTER-WORD
SB-INT:SIMPLE-STYLE-WARNING: undefined function: KFTT-DATA-PARSER:CALC-DATA-SIZE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: KFTT-DATA-PARSER:CALC-MAX-LENGTH


Unable to find framework CUDA


## Seq2Seqを用いてKFTTを学習

cl-waffeのRNNを用いて[京都フリー翻訳タスク (KFTT)](http://www.phontron.com/kftt/index-ja.html#leaders)を学習させる例です。

学習データのダウンロード (0.4GB程度の容量が必要です。)
```shell
$ curl -O http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
$ tar xzvf kftt-data-1.0.tar.gz
```

In [3]:
(defpackage :seq2seq-sin
    (:use :cl :cl-waffe :cl-waffe.nn :clgplot :mgl-mat :kftt-data-parser))

(in-package :seq2seq-sin)

#<PACKAGE "SEQ2SEQ-SIN">

#<PACKAGE "SEQ2SEQ-SIN">

### Seq2Seq

Seq2Seq

問題：

### Seq2Seqモデルを定義する。

In [31]:
(defmodel Encoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1)))
  :forward ((x)
            (with-calling-layers x
                (embedding x)
                (layer x))))

(defmodel Decoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1))
               (h2l       (linearlayer hidden-size vocab-size)))
  :forward ((encoder-state y)
            (let* ((ye (call (self embedding) y))
                   (hs (call (self layer) ye encoder-state))
                   (h-output (call (self h2l) hs)))
                  (list h-output hs))))

(defmodel Seq2Seq (vocab-size-lang1 vocab-size-lang2 embedding-dim input-size)
  :parameters ((encoder (Encoder vocab-size-lang1 embedding-dim input-size))
               (decoder (Decoder vocab-size-lang2 embedding-dim input-size)))  
  :forward ((x y)
            (let ((x-state (call (self encoder) x))
                  (y1 (!zeros (!shape y))))
                 (setq y1 (setf (!aref y1 t) (!aref y '(1 t))))
                 (call (self decoder) x-state y1))))


NIL

NIL

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::COPY-ENCODER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::ENCODER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::ENCODER-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::ENCODER-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::ENCODER-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::ENCODER-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::ENCODER-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::ENCODER-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::ENCODER-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::ENCODER-OBJECT-TYPE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF

### パラメーターの設定

In [123]:
(defparameter lang1 :ja)
(defparameter lang2 :en)

(defparameter batch-size 64)
(defparameter epoch-num 10)
(defparameter lr 1e-3)
(defparameter embedding-dim 64)
(defparameter hidden-size 128)
; ja -> en

LANG1

LANG2

BATCH-SIZE

EPOCH-NUM

LR

EMBEDDING-DIM

HIDDEN-SIZE

In [124]:
(deftrainer Seq2SeqTrainer (vocab-size-lang1 vocab-size-lang2 embedding-dim hidden-size)
  :model (Seq2Seq vocab-size-lang1 vocab-size-lang2 embedding-dim hidden-size)
  :optimizer cl-waffe.optimizers:Adam
  :optimizer-args (:lr lr)
  :step-model ((x y)
               (zero-grad)
               (let* ((outs (call (self model) x y))
                      (out (softmax-cross-entropy (car outs) y)))
                     (backward out)
                     (update)
                     out))
  :predict ((x y) (call (model) x y)))

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::COPY-SEQ2SEQTRAINER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::SEQ2SEQTRAINER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::SEQ2SEQTRAINER-MODEL) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::SEQ2SEQTRAINER-MODEL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::SEQ2SEQTRAINER-OPTIMIZER) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::SEQ2SEQTRAINER-OPTIMIZER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::SEQ2SEQTRAINER-OPTIMIZER-REPORT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::SEQ2SEQTRAINER-OPTIMIZER-REPORT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::SEQ2SEQTRAINER-PREDICT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::SEQ2SEQTRAINER-PREDICT in DEFUN
SB-KE

### データの前処理

語彙数削減のためSentencePieceを用いる（TODO)

文字数でソートしてPaddingのコストを減らす（TODO)

In [125]:
; 辞書データを作る
(multiple-value-bind (lang1-w2i lang1-i2w) (collect-tokens :dev lang1 T)
    (defparameter w2i-lang1 lang1-w2i)
    (defparameter i2w-lang1 lang1-i2w))

(multiple-value-bind (lang2-w2i lang2-i2w) (collect-tokens :dev lang2 T)
    (defparameter w2i-lang2 lang2-w2i)
    (defparameter i2w-lang2 lang2-i2w))

;w2i: Hash Table, 単語(英語+日本語) -> Index, i2w: Index -> 単語

(format nil "Vocab_Size: ~% Lang1: ~a ~% Lang2: ~a" (hash-table-count w2i-lang1) (hash-table-count w2i-lang2))

I2W-LANG1

I2W-LANG2

"Vocab_Size: 
 Lang1: 4519 
 Lang2: 4780"

In [126]:
; 文字列操作のユーティリティ

(defun split (x str)
  (let ((pos (search x str))
        (size (length x)))
    (if pos
      (cons (subseq str 0 pos)
            (split x (subseq str (+ pos size))))
      (list str))))

(defun list-to-tensor (list)
    (const (make-mat (length list) :initial-contents list)))

(defun words-into-idx (words lang)
    "words: 単語のList, (e.g.: 臨済 宗 （ 臨濟 宗 、)... )
     lang : :ja or :en"
    
    (list-to-tensor
     (map 'list #'(lambda (word)
                          (cond
                              ((eql lang lang1)
                               (or (gethash word w2i-lang1)
                                   (gethash "<UNK>" w2i-lang1)))
                              ((eql lang lang2)
                               (or (gethash word w2i-lang1)
                                   (gethash "<UNK>" w2i-lang2)))
                              (T (error "")))) `("<BOS>" ,@words "<EOS>"))))

(defun make-train-data (language data-type)
    "Language: 言語
     data-type: :train or :test"
    
    (let ((result nil))
         (time (with-open-kftt-file line :tok data-type language
                   (push
                    (words-into-idx (split " " line) language)
                    result)))
         (reverse result)))

SPLIT

LIST-TO-TENSOR

WORDS-INTO-IDX

MAKE-TRAIN-DATA

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::SPLIT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::LIST-TO-TENSOR in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::WORDS-INTO-IDX in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::MAKE-TRAIN-DATA in DEFUN


In [127]:
(defparameter train_X (make-train-data lang1 :dev))

TRAIN_X

Evaluation took:
  0.023 seconds of real time
  0.016254 seconds of total run time (0.015903 user, 0.000351 system)
  69.57% CPU
  53,786,360 processor cycles
  10,447,744 bytes consed
  


In [128]:
(defparameter train_Y (make-train-data lang2 :dev))

TRAIN_Y

Evaluation took:
  0.020 seconds of real time
  0.018860 seconds of total run time (0.018431 user, 0.000429 system)
  95.00% CPU
  46,825,860 processor cycles
  14,849,872 bytes consed
  


In [129]:
(defparameter test_X (make-train-data lang1 :dev))
(defparameter test_Y (make-train-data lang2 :dev))

TEST_X

TEST_Y

Evaluation took:
  0.046 seconds of real time
  0.036809 seconds of total run time (0.022714 user, 0.014095 system)
  80.43% CPU
  106,286,040 processor cycles
  10,446,096 bytes consed
  
Evaluation took:
  0.057 seconds of real time
  0.045780 seconds of total run time (0.026860 user, 0.018920 system)
  80.70% CPU
  132,865,464 processor cycles
  14,844,528 bytes consed
  


### データセットを定義する



In [155]:
(defdataset KFTT-Dataset (x y batch-size)
    :parameters ((x x)
                 (y y)
                 (batch-size batch-size))
    :next ((index)
           "Returns (list X[index:(+ index batch-size)] Y[index:(+ index batch-size)])"
           (let* ((batch-size (self batch-size))
                  (end (min (length (self x)) (+ index (self batch-size))))
                  (x (loop for i fixnum upfrom index below end
                           collect (nth i (self x))))
                  (y (loop for i fixnum upfrom index below end
                           collect (nth i (self y))))
                  (batch-maxlen (max (apply #'max (map 'list #'!size x))
                                     (apply #'max (map 'list #'!size y))))
                  (result-x (!zeros `(,(length x) ,batch-maxlen)))
                  (result-y (!zeros `(,(length y) ,batch-maxlen))))
                 
                 (dotimes (index (1- (the fixnum (length x))))
                     (setf (!aref result-x index `(0 ,(!size (nth index x)))) (nth index x))
                     (setf (!aref result-y index `(0 ,(!size (nth index y)))) (nth index y)))
                 (list result-x result-y)))
    :length (() (length (self x))))

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::COPY-KFTT-DATASET in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::KFTT-DATASET-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::KFTT-DATASET-X) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::KFTT-DATASET-X in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::KFTT-DATASET-Y) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::KFTT-DATASET-Y in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::KFTT-DATASET-BATCH-SIZE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::KFTT-DATASET-BATCH-SIZE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-SIN::KFTT-DATASET-LENGTH) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-SIN::KFTT-DATASET-LENGTH in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF

In [156]:
(defparameter *train-dataset* (KFTT-Dataset train_x train_y batch-size))

(time (dotimes (i 100) (get-dataset *train-dataset* 0)))

*TRAIN-DATASET*

NIL

Evaluation took:
  0.184 seconds of real time
  0.132699 seconds of total run time (0.104879 user, 0.027820 system)
  72.28% CPU
  425,693,716 processor cycles
  35,813,760 bytes consed
  


In [157]:
(defparameter model (seq2seqtrainer (hash-table-count w2i-lang1)
                                    (hash-table-count w2i-lang2)
                                    embedding-dim
                                    hidden-size))
model

MODEL

<Trainer: SEQ2SEQTRAINER()>

In [165]:
(defparameter *start-time* (get-universal-time))

(dotimes (epoch epoch-num)
    (format t "~ath Epoch: [~as]~%" epoch (- (get-universal-time) *start-time*))
    (let ((dataset-size (get-dataset-length *train-dataset*)))
         (loop for batch-num fixnum upfrom 0 below dataset-size by batch-size
               do (let ((datum (get-dataset *train-dataset* batch-num)))
                       (print datum)))))

*START-TIME*

NIL

0th Epoch: [0s]

(#Const(((3.0 5.0 ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 86) :backward NIL)
 #Const(((3.0 2.0 ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 86) :backward NIL)) 
(#Const(((3.0 463.0... ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 52) :backward NIL)
 #Const(((3.0 2.0 ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 52) :backward NIL)) 
(#Const(((3.0 664.0... ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 112) :backward NIL)
 #Const(((3.0 2.0 ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 112) :backward NIL)) 
(#Const(((3.0 910.0... ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl t :shape (64 97) :backward NIL)
 #Const(((3.0 2.0 ~ 0.0 0.0)        
                 ...
        (0.0 0.0 ~ 0.0 0.0)) :mgl 